## Reporting Minimization

You can report the status of [`simulation.minimizeEnergy()`](http://docs.openmm.org/development/api-python/generated/openmm.app.simulation.Simulation.html?#openmm.app.simulation.Simulation.minimizeEnergy) using a [`MinimizerReporter`](http://docs.openmm.org/development/api-python/generated/openmm.openmm.MinimizationReporter.html).

The syntax for doing this is somewhat different to typical OpenMM functionality. Read the [API documentation](http://docs.openmm.org/development/api-python/generated/openmm.openmm.MinimizationReporter.html) for more explanation.

First we will create a test system.

In [ ]:
from openmm.app import *
from openmm import *
from openmm.unit import *
from sys import stdout

pdb = PDBFile('villin.pdb')
forcefield = ForceField('amber14-all.xml', 'amber14/tip3pfb.xml')
system = forcefield.createSystem(pdb.topology, nonbondedMethod=PME,
        nonbondedCutoff=1*nanometer, constraints=HBonds)
integrator = LangevinMiddleIntegrator(300*kelvin, 1/picosecond, 0.004*picoseconds)
simulation = Simulation(pdb.topology, system, integrator)
simulation.context.setPositions(pdb.positions)

We then create a `MinimizationReporter` class that will print the current energy to the screen every 100 steps and save the energies to an array which we can plot later

In [ ]:
# Define a subclass of MinimizationReporter
class Reporter(MinimizationReporter):
    interval = 100 # report interval
    energies = [] # array to record progress
    def report(self, iteration, x, grad, args):
        # print current system energy to screen 
        if iteration % self.interval == 0:
            print(iteration, args['system energy'])

        # save energy at each iteration to an array we can use later
        self.energies.append(args['system energy'])

        # The report method must return a bool specifying if minimization should be stopped. 
        # You can use this functionality for early termination.
        return False

We now create an instance of the reporter and minimize the system with the reporter attached.

In [ ]:
# Create an instance of our reporter
reporter = Reporter()

# Perform local energy minimization 
simulation.minimizeEnergy(reporter=reporter)

# plot the minimization progress
import matplotlib.pyplot as plt
plt.plot(Reporter.energies)
plt.show()